In [1]:
from keras.models import load_model
import os
import keras.backend.tensorflow_backend as tb
tb._SYMBOLIC_SCOPE.value = True
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from keras_facenet import FaceNet


import time
import json
import mysql.connector
from SocialProcessor import SocialProcessor

import vk_api
import threading

%matplotlib inline

Using TensorFlow backend.


In [2]:
class StoppableThread(threading.Thread):
    """
    Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition.
    
    """
    func = None
    args = {}

    def __init__(self,  *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    def run(self):
        self.args['thread'] = self
        self.func(**self.args)
    

In [3]:
embedder = FaceNet()

In [4]:
def aunt_vk():
    return input(), True


def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [5]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [6]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

In [7]:
def performVkProcessing(delay=0, count=1, thread=False):
    conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")
    processor = SocialProcessor(conn, embedder, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/', batch=600, limit=500)
    time.sleep(delay)
    while count>0:
        if type(thread) != type(False):
            if thread.stopped():
                break
        task = processor.getTask()
        user_id = task[0]
        query_id = task[-1]
        processor.delTask(query_id)
        
        processor.processVkUser(api, user_id)
        count-=1

In [17]:
performVkProcessing(0,1)

Procesing 185313244


Empty face: https://sun9-38.userapi.com/c623229/v623229244/20643/_lGnL4928qs.jpg (2536, 2478, 1280, 960)
Empty face: https://sun9-27.userapi.com/c633920/v633920244/13421/a-5KXS6VtY8.jpg (1213, 1176, 604, 604)
Empty face: https://sun9-6.userapi.com/c841630/v841630612/de70/mOyly6qz1ls.jpg (148, 143, 75, 69)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (142, 141, 75, 75)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (207, 143, 75, 75)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (149, 214, 75, 75)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (149, 83, 75, 75)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (80, 143, 75, 75)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (215, 82, 75, 75)
Empty face: https://sun9-30.userapi.com/c834303/v834303111/197a0/clzhyQWEMxY.jpg (218, 219, 75, 

In [16]:
threads = []
threading.active_count()

5

In [19]:
def addThreads(current_num, max_num=100):
    t_num = max(0, max_num - current_num)
    limits = 5
    step = 1.5
    params = [(i*step, limits) for i in range(t_num)]
    print("Added %s threads"% (len(params)))
    for args in params:
        x = StoppableThread()
        x.func = performVkProcessing
        args = {"delay": args[0], "count": args[1]}
        x.args = args
        x.start()
        threads.append(x)

In [21]:
while True:
    active = sum([i.isAlive() for i in threads])
    print("Active threads:", active)
    addThreads(active)
    active = sum([i.isAlive() for i in threads])
    print("Active threads:", active)
    i = 0
    while i < len(threads):
        if threads[i].isAlive():
            i += 1
        else:
            threads.pop(i)
    time.sleep(30)


Active threads: 98
Added 2 threads
Active threads: 100
Procesing 261510331


Procesing 104931636


209291960:



Added 95 rows
Procesing 483601390



Active threads: 100
Added 0 threads
Active threads: 100


361851730:

Added 8 rows
Procesing 246272986


483601390:
Added 2 rows
Procesing 18166848


...


133855831:
Added 71 rows
Procesing 125933237



155438905:
Active threads: 100
Added 0 threads
Active threads: 100
102980906:
24022895:
137935648:
Added 22 rows
160828202:
302285557:
Added 1 rows


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-b965d3415cf8>", line 13, in <module>
    time.sleep(30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_o


KeyboardInterrupt



In [41]:
while i < len(threads):
        if threads[i].isAlive():
            i += 1
        else:
            threads.pop(i)
sum([i.isAlive() for i in threads])

4

In [43]:
# conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")
# processor = SocialProcessor(conn, model, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/', batch=512)
# processor.processVkUser(api, 1422)

In [23]:
[i.stop() for i in threads]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Procesing 114684303


Procesing 280900435


Added 274 rows
